In [1]:
%pip install sentence-transformers
%pip install polars
%pip install torch
%pip install datasets
%pip install -U transformers[torch]
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: transformers[torch]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Importing Libraries

In [ ]:
import polars as pl
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader
from sentence_transformers import InputExample
import matplotlib.pyplot as plt

/Users/michael/Documents/School/Year5/COS802/Project/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Data Processing 

In [3]:
def remove_punctuation(df, col):
    return df.with_columns(
        pl.col(col).str.replace_all(r"[^\w\s]", "")
    )

In [4]:
def lowercase(df, col):
    return df.with_columns(
        pl.col(col).str.to_lowercase()
    )

# Reading the Data

In [5]:
def dataset(lang, test=False):
    if test:
        file_path = f"hf://datasets/SemRel/SemRel2024/{lang}/test-*"
    else:
        file_path = f"hf://datasets/SemRel/SemRel2024/{lang}/dev-*"
    return pl.read_parquet(file_path)

In [6]:
def read_datasets():
    languages = [
        "afr",
        "arq",
        "amh",
        "eng",
        "hau",
        "ind",
        "hin",
        "kin",
        "mar",
        "arb",
        "ary",
        "pan",
        "esp",
        "tel",
    ]

    datasets_train = {}
    for lang in languages:
        # Load language-specific dataset
        
        df = dataset(lang)
        df = remove_punctuation(df, 'sentence1')
        df = remove_punctuation(df, 'sentence2')
        df = lowercase(df, 'sentence1')
        df = lowercase(df, 'sentence2')

        datasets_train[lang] = df

    return datasets_train

def read_dataset():
    df = dataset('*')
    df = remove_punctuation(df, 'sentence1')
    df = remove_punctuation(df, 'sentence2')
    df = lowercase(df, 'sentence1')
    df = lowercase(df, 'sentence2')

    return df

# Using SBERT

In [7]:
def train_model(train_data, batch_size=32, num_epochs=5):
    training = []
    for row in train_data.iter_rows(named=True):
        training.append(InputExample(
            texts=[row['sentence1'], row['sentence2']],
            label=row['label']
        ))

    # Load the multilingual SBERT model
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    print(model.device)

    # Loss function
    train_loss = losses.CosineSimilarityLoss(model=model)

    # Wrap the examples in a DataLoader
    train_dataloader = DataLoader(training, shuffle=True, batch_size=batch_size)

    # Lists to track loss over epochs
    epoch_losses = []

    # Training loop with loss tracking
    for epoch in range(num_epochs):
        model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=num_epochs,  # Train for one epoch each loop iteration
            warmup_steps=100,
            show_progress_bar=True
        )
        # Record the average loss for this epoch
        avg_loss = sum(train_loss.losses) / len(train_loss.losses)  # Average loss for this epoch
        epoch_losses.append(avg_loss)
        train_loss.losses.clear()  # Reset the loss list for the next epoch

    return epoch_losses

In [8]:
data = read_dataset()

In [9]:

epoch_losses = train_model(data, batch_size=8)
# Plotting the training loss
plt.plot(range(1, 6), epoch_losses, marker='o')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Average Training Loss')
plt.grid(True)
plt.show()

mps:0


  6%|▌         | 87/1545 [00:35<08:48,  2.76it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 3.27 GB, other allocations: 14.71 GB, max allowed: 18.13 GB). Tried to allocate 366.27 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).